```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Basic Particle Filter with the complete model

This notebook implements an experiment of using a basic particle filter to assimilate information obtained from a Lagrangian drifter into the Eulerian ocean field.

### A particle
Each particle is an ocean model, which conceptually don't hold any drifters. Drifters are displayed in the simulations in order to show how each particle differs in the animations. The state vector, however, is purely based on 
$$\psi_i^n = [\eta_i^n, hu_i^n, hv_i^n]^T \in \mathbb{R}^{3 n_x n_y}$$


### The model
The model is a shallow water model solved with the CDKLM scheme. Drifter intergration is used in the syntetic truth, and in order to visualize differences between particles.
A small-scale stochastic term is added to each particle for every timestep, to represent the model error.


### The truth
A syntetic truth is used in the form of an identic twin.

### The observation
The observations are based on how the Lagrangian drifters change positions between observations.
This change represent a velocity, so that $y^n = [hu_{j,k, truth}^n, hv_{j,k, truth}^n]^T$, in which $(j,k)$ represent the index of the cell where the latest observation was made.

### Resampling
We could here apply any of the four resampling schemes, but will use *Residual Resampling* as a first / default method.
Since we have a stochastic model, we will use exact duplications within the resampling scheme. 

## Why walk straight into the curse of dimensionallity?

As we know (see Peter Jan's example from 2015, and also shown during the data assimilation course in March 2018), the curse of dimensionality will make the basic particle filters break at once in the presence of high-dimensional systems. 

What was realized during the preparation of my (Havard's) exam in *Techniques in Data Assimilation*, high-dimensionality here referes to the observation space, and not the state space!!! The basic particle filters should therefore (in theory, possibly) work completely fine on high-dimensional problems, as long as the observations are kept low-dimensional.

In our case here, as long as we observe few drifters, we will have low-dimensional observation vectors, and the curse of dimensionality does not apply.


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import os
import sys
from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as dautils


In [ ]:
%cuda_context_handler gpu_ctx

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- drift a drifter
- applies a localized small-scale error
- observes the drifter position

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


## Create initial condition for ensemble:

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
initOption = 3
if initOption == 1:
    # Original initial conditions
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 2:
    # Initial conditions used for the SIR filter
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 3:
    # Initial conditions random - see further down!
    pass
    

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print ("q0: ", q0)
print ("[f, g, H]", [f, g, waterDepth])
print ("f/gH: ", f/(g*waterDepth))
print ("gH/f: ", g*waterDepth/f)

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)
if initOption == 3:
    sim.perturbState(q0_scale=100)

ensemble_size = 40
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(gpu_ctx, ensemble_size, sim,
                                                 num_drifters=9,
                                                 observation_type=dautils.ObservationType.UnderlyingFlow,
                                                 #observation_type=dautils.ObservationType.DirectUnderlyingFlow,
                                                 observation_variance = 0.01,#**2,
                                                 initialization_variance_factor_ocean_field=20)

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble, trueStateOnly=True)

T = 100
#T = 35
sub_t = 100*dt
resampling_points = [36, 72, 108]
#resampling_points = [9, 18, 27, 36, 45, 56]
resampling_points = range(5, 100, 10)
print( "Will resample at iterations: ", resampling_points)
infoPlots = []

def animate(i):
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0

    for rp in resampling_points:
        if i == rp:
            print ("resampling at iteration " + str(i))
            infoFig = ensemble.plotDistanceInfo(title="it = " + str(i) + " before resampling")
            plt.close(infoFig)
            infoPlots.append(infoFig)
            
            dautils.residualSampling(ensemble)
            
            infoFig = ensemble.plotDistanceInfo(title="it = " + str(i) + " post resampling")
            plt.close(infoFig)
            infoPlots.append(infoFig)
    
    plotter.plot(ensemble);
    
    ensemble.getEnsembleVarAndRMSEUnderDrifter(i)

    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print( "{:03.0f}".format(100*i / T) + " % => t=" + str(t) )

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
ensemble.plotEnsemble()

In [ ]:
max_dt = ensemble.findLargestPossibleTimeStep()
print ("Largest possible timestep with this case: ", max_dt)

In [ ]:
def show_figures(figs):
    for f in figs:
        dummy = plt.figure()
        new_manager = dummy.canvas.manager
        new_manager.canvas.figure = f
        f.set_canvas(new_manager.canvas)
        filename= f._suptitle.get_text().replace(" ", "_").replace("=_", "") + ".png"
        #plt.savefig(filename)
show_figures(infoPlots)
fig = ensemble.plotDistanceInfo(title="Final ensemble, it = " + str(T))

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.05*np.ones_like(resampling_points), 'o')
plt.title("RMSE under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.1*np.ones_like(resampling_points), 'o')
plt.title("Std.dev under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 1.0*np.ones_like(resampling_points), 'o')
plt.title("r = var/rmse under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 5])

print( np.sqrt(ensemble.observation_cov[0,0]))

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.05*np.ones_like(resampling_points), 'o')
plt.title("RMSE under drifter")
plt.legend(loc=0)
plt.grid()

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.05*np.ones_like(resampling_points), 'o')
plt.title("Variance under drifter")
plt.legend(loc=0)
plt.grid()

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 1.0*np.ones_like(resampling_points), 'o')
plt.title("r = var/rmse under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0.2, 1.1])
